# Importing required modules and programs

In [ ]:
from dbconnector import connection
from client import Client
import time
import threading
import hashlib
import os

# Connecting To Database

In [1]:
mycursor,db = connection("192.168.1.11")
print("[SUCCESS] Connected to database succesfully...")

# Making query in Database

In [3]:
Filename = input("Enter Filename: ")

print("[SEARCHING] Searching for file in database...")

mycursor.execute(f"""SELECT u.PublicIP, f.SIZE, f.CHECKSUM FROM filedatabase f INNER JOIN userdatabase u USING (USERNAME) 
WHERE f.FILENAME = '{Filename}' """)

listof_IPs = []

size = 0
checksum = ""

for x in mycursor:
    listof_IPs.append(x[0])
    size = int(x[1])
    checksum = x[2]
print("[COMPLETE] Search Complete...")

# Connecting to peers

In [9]:
number_of_peers = len(listof_IPs)
inc = size//number_of_peers
offset = 0
offset_lookup = {}
for i,IP in enumerate(listof_IPs):
    if IP not in offset_lookup:
        offset_lookup[IP] = dict()
    offset_lookup[IP]['start'] = offset
    if i == number_of_peers-1:
        offset_lookup[IP]['end'] = size
    else:
        offset_lookup[IP]['end'] = offset + inc
        offset += inc
client_lookup = {}
for IP in listof_IPs:
    client_lookup[IP] = Client(IP,offset_lookup[IP]['start'],offset_lookup[IP]['end'],Filename)
    
for IP in client_lookup:
    client_lookup[IP].handshake()



Lists T1.txt with size 5847 Found on SERVER! 
Lists T1.txt with size 5847 Found on SERVER! 


# Receiving Chunks

In [7]:
location = input("Enter Location where you want to download file: ")

tick = time.time()

print("[DOWNLOADING] Started downloading...")
threads=[]
for IP in client_lookup:
    t=threading.Thread(target=client_lookup[IP].receiving, args=[location])
    t.start()
    threads.append(t)
for thread in threads:
    thread.join()

clock= time.time()
print(f"[COMPLETED] Finished downloading in {(clock-tick):.2f} seconds!")

Started receiving
Started receiving
130990
1917010
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
330340
37960
2048000
189800
138700
211700
140160
2048000
100740
27740
112420
135780
54020
2048000
2048000
569400
211700
2048000275940

128480
162060
236520
20480004380

186880
153300
5840
1868802048000

344560
52560
46720
3095202048000

140160
54020
277400
2048000
40880
96360
2048000
2048000
2048000
2048000
1181140
2048000
378140
265720
2048000
2048000
1404520
2744802048000

347480
2048000
2048000
2048000
2048000
1426420
2048000256960

343100
2048000
2048000
2048000
2048000
1447120
2920
94900
2048000
2048000
2048000
1950560
195640
2048000
2048000
2048000
2048000
2048000
2048000
2048000
2048000
1852740
160600
1010249
Receiving complete from 192.168.1.11
111602249
1886640
24820
2023560
138700
1909680
189800
1857440
335800
159140
80300
37960
1435180
259880
1788500
375220
157680
121180
1394300
71540
197538

# Joining Chunks

In [14]:
print("[JOINING] Joining chunks...")
target = Filename
count = 0
for IP in client_lookup:
    with open(IP,'rb') as tempfile:
        data  = tempfile.read()
        with open(target,'ab') as dest:
            dest.write(data)
    count += 1
    print(f"{count} of {number_of_peers} chunks joined...")
    os.remove(IP)
print("[COMPLETE] Joining complete...")

# Performing Checksum

In [48]:
print("[VERIFYING] Verfiying download...")

hasher = hashlib.md5()

with open(target,'rb') as Source:
    content = Source.read()
    hasher.update(content)
recvd_check=hasher.hexdigest()

if recvd_check != checksum:
     print("[FAILED] Please try again...")
else:
     print("[SUCCESSFUL] Downloaded Succesfully...")

failed


In [49]:
print(source_check)

297e7d12b2b64441cd929082982020db


In [50]:
print(dest_check)

5542901df1647fc9a9083f796fac4364
